#Rarefaction, beta-diversity analysis and alpha-diversity calculations

In [1]:
#Import packages
library(phyloseq)
library(vegan)
library(ggplot2)
library(pspearman)
library(OneR)
library(plyr)
library(dplyr)

Loading required package: permute
Loading required package: lattice
This is vegan 2.5-7

Attaching package: ‘dplyr’

The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
#import phyloseq object with 16s count data and metadata
data<-readRDS('statins_16S.rds')

In [5]:
#Rarefy the dataset to even depth
print(min(sample_sums(data)))
rarefied_data=rarefy_even_depth(data, sample.size = min(sample_sums(data)),
  rngseed = 321, replace = FALSE, trimOTUs = TRUE, verbose = TRUE)
#save the rarefied phyloseq object file
saveRDS(rarefied_data,'rarefied_16s.rds')

[1] 25596


`set.seed(321)` was used to initialize repeatable random subsampling.
Please record this for your records so others can reproduce.
Try `set.seed(321); .Random.seed` for the full vector
...
51614OTUs were removed because they are no longer 
present in any sample after random subsampling

...


In [6]:
rarefied_data<-readRDS('rarefied_16s.rds')
rarefied_data

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 37408 taxa and 1512 samples ]
sample_data() Sample Data:       [ 1512 samples by 32 sample variables ]
tax_table()   Taxonomy Table:    [ 37408 taxa by 8 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 37408 tips and 35496 internal nodes ]

In [8]:
rarefied_phylum<-tax_glom(rarefied_data,"Phylum")

In [9]:
rarefied_genus<-tax_glom(rarefied_data,"Genus")

In [10]:
saveRDS(rarefied_genus,'rarefied_genus_16S.rds')

In [11]:
rarefied_genus<-readRDS('rarefied_genus_16S.rds')

In [ ]:
#Perform beta-diversity analysis on the bray-curtis dissimilarity matrix
set.seed(123)
metadata <- as(sample_data(rarefied_genus), "data.frame")
#calculate distance matrix
bray<-distance(rarefied_genus, method="bray")
#test for association between interindividual gut microbiome variation and statin use, statin*HMG interaction
adonis(bray~ Group*HMG,
       data = metadata,permutations = 3000)
adonis(bray ~ vendor_microbiome+BMI+sex+age+Group*HMG,
       data = metadata,permutations = 3000)

In [ ]:
#Perform beta-diversity analysis on the Weighted Unifrac dissimilarity matrix
set.seed(123)
wunifrac<-distance(rarefied_genus, method="wunifrac")
#test for association between interindividual gut microbiome variation and statin use
adonis(wunifrac~ Group*HMG,
       data = metadata,permutations = 3000)
adonis(wunifrac~ vendor_microbiome+BMI+sex+age+Group*HMG,
       data = metadata,permutations = 3000)

In [ ]:
rarefied_genotek<-readRDS('rarefied_16s.rds')
#calculate alpha-diversity measures rarefied using ASV count data
richness_df<-estimate_richness(rarefied_genotek,measures=c('Observed','Shannon'))

In [ ]:
#add participant id to the df
richness_df$public_client_id<-sample_data(rarefied_genotek)$public_client_id

In [ ]:
#export alpha-diversity dataframe for further statistical analysis
write.csv(richness_df,'richness_df.csv')